In [1]:
import networkx as nx
import numpy as np

import scipy.optimize
import scipy.sparse

from networkx.utils import np_random_state

from src.python._process_params import _process_params
from src.python.getMatrix import getMatrixByName
from src.python.vis.visGraph import visGraph

In [2]:
# Copied from networkx.drawing.layout.py
@np_random_state(10)
def spring_layout(
    G,
    k=None,
    pos=None,
    fixed=None,
    iterations=50,
    threshold=1e-4,
    weight="weight",
    scale=1,
    center=None,
    dim=2,
    seed=None,
    method="FR",  # ! added method = "FR" or "RS"
):
    G, center = _process_params(G, center, dim)

    if fixed is not None:
        if pos is None:
            raise ValueError("nodes are fixed without positions given")
        for node in fixed:
            if node not in pos:
                raise ValueError("nodes are fixed without positions given")
        nfixed = {node: i for i, node in enumerate(G)}
        fixed = np.asarray([nfixed[node] for node in fixed if node in nfixed])

    if pos is not None:
        # Determine size of existing domain to adjust initial positions
        dom_size = max(coord for pos_tup in pos.values() for coord in pos_tup)
        if dom_size == 0:
            dom_size = 1
        pos_arr = seed.rand(len(G), dim) * dom_size + center

        for i, n in enumerate(G):
            if n in pos:
                pos_arr[i] = np.asarray(pos[n])
    else:
        pos_arr = None
        dom_size = 1

    if len(G) == 0:
        return {}
    if len(G) == 1:
        return {nx.utils.arbitrary_element(G.nodes()): center}

    # ! Changed a lot
    A = nx.to_scipy_sparse_array(G, weight=weight, dtype="f")
    if k is None and fixed is not None:
        # We must adjust k by domain size for layouts not near 1x1
        nnodes, _ = A.shape
        k = dom_size / np.sqrt(nnodes)
    return _sparse_fruchterman_reingold(
        A, k, pos_arr, fixed, iterations, threshold, dim, seed, method
    )

In [24]:
from networkx.utils import np_random_state
from src.python.cost import cost
import scipy as sp
from typing import Tuple


# Copied from networkx.drawing.layout.py
@np_random_state(7)
def _sparse_fruchterman_reingold(
    A,
    k=None,
    pos=None,
    fixed=None,
    iterations=50,
    threshold=1e-4,
    dim=2,
    seed=None,
    method="FR",
    verbose=True,
):
    from scipy.optimize import minimize

    try:
        nnodes, _ = A.shape
    except AttributeError as err:
        msg = "fruchterman_reingold() takes an adjacency matrix as input"
        raise nx.NetworkXError(msg) from err

    if pos is None:
        # random initial positions
        pos = np.asarray(seed.rand(nnodes, dim), dtype=A.dtype)
    else:
        # make sure positions are of same type as matrix
        pos = pos.astype(A.dtype)

    # no fixed nodes
    if fixed is None:
        fixed = []

    # optimal distance between nodes
    if k is None:
        k = np.sqrt(1.0 / nnodes)

    if method == "FR":
        # make sure we have a LIst of Lists representation
        try:
            A = A.tolil()
        except AttributeError:
            A = (sp.sparse.coo_array(A)).tolil()

        # the initial "temperature" is about .1 of domain area (=1x1)
        # this is the largest step allowed in the dynamics.
        t = max(max(pos.T[0]) - min(pos.T[0]), max(pos.T[1]) - min(pos.T[1])) * 0.1
        # simple cooling scheme.
        # linearly step down by dt on each iteration so last iteration is size dt.
        dt = t / (iterations + 1)

        displacement = np.zeros((dim, nnodes))
        for iteration in range(iterations):
            displacement *= 0
            # loop over rows
            for i in range(A.shape[0]):
                if i in fixed:
                    continue
                # difference between this row's node position and all others
                delta = (pos[i] - pos).T
                # distance between points
                distance = np.sqrt((delta**2).sum(axis=0))
                # enforce minimum distance of 0.01
                distance = np.where(distance < 0.01, 0.01, distance)
                # the adjacency matrix row
                Ai = A.getrowview(i).toarray()  # TODO: revisit w/ sparse 1D container
                # displacement "force"
                displacement[:, i] += (
                    delta * (k * k / distance**2 - Ai * distance / k)
                ).sum(axis=1)
            # update positions
            length = np.sqrt((displacement**2).sum(axis=0))
            length = np.where(length < 0.01, 0.1, length)
            delta_pos = (displacement * t / length).T
            yield pos, delta_pos
            pos += delta_pos
            # cool temperature
            t -= dt
            if verbose:
                print(f"{cost(pos,A,k)=}")
            if (np.linalg.norm(delta_pos) / nnodes) < threshold:
                break
    elif method == "L-BFGS-B" or method == "BFGS" or method == "CG":
        # make sure we have a coo_matrix representation
        try:
            A = A.tolil()
        except AttributeError:
            A = (sp.sparse.coo_array(A)).tolil()

        def cost_fun(x):
            EPS = 1e-10
            pos = x.reshape((nnodes, dim))
            grad = np.zeros((nnodes, dim))
            cost = 0.0
            for i in range(nnodes):
                Ai = A.getrow(i).toarray().flatten()
                delta = pos[i] - pos
                assert np.all(delta[i] == 0.0)
                distance = np.linalg.norm(delta, axis=1)
                cost += np.sum(
                    Ai * distance**3 / (3 * k) - (k**2) * np.log(distance + EPS)
                )
                distance = np.where(distance < 0.01, 0.01, distance)
                distance_inv = 1 / distance
                coefficient1 = Ai * distance / k - (k * distance_inv) ** 2
                grad[i] = coefficient1 @ delta
            # print(f"{cost=}")
            return cost, grad.ravel()

        pos_hist = []
        res = sp.optimize.minimize(
            cost_fun,
            pos.ravel(),
            method=method,
            jac=True,
            options={"maxiter": iterations, "disp": verbose},
            callback=lambda x: pos_hist.append((x.reshape((nnodes, dim)))),
        )

        for i in range(len(pos_hist) - 1):
            yield pos_hist[i], pos_hist[i + 1] - pos_hist[i]
            # yield pos_hist[i], cost_fun(pos_hist[i])[1].reshape((nnodes, dim))

        if verbose:
            print("         WarnFlag and message: %d" % res.status, res.message)
            print("         Current function value: %f" % res.fun)
            print("         Iterations: %d" % res.nit)

    elif method == "coordinate_descent":
        from scipy.optimize import line_search

        # make sure we have a coo_matrix representation
        try:
            A = A.tolil()
        except AttributeError:
            A = (sp.sparse.coo_array(A)).tolil()

        new_pos = np.copy(pos)

        # todo fixed nodes
        for _ in range(iterations):
            grads = np.zeros((nnodes, dim))
            for i in range(nnodes):
                Ai = A.getrow(i).toarray().flatten()
                delta = pos[i] - pos
                assert np.all(delta[i] == 0.0)
                distance = np.linalg.norm(delta, axis=1)
                distance = np.where(distance < 1e-5, 1e-5, distance)
                grad = (Ai * distance / k - (k / distance) ** 2) @ delta
                grads[i] = grad

                cache = dict()

                def cost_fun(xi: np.ndarray) -> float:
                    if xi.tobytes() in cache:
                        return cache[xi.tobytes()]
                    delta = xi - pos
                    delta[i].fill(0.0)
                    distance = np.linalg.norm(delta, axis=1)
                    distance = np.where(distance < 1e-5, 1e-5, distance)
                    cost = np.sum(
                        Ai * distance**3 / (3 * k) - (k**2) * np.log(distance)
                    )
                    distance = np.where(distance < 0.01, 0.01, distance)
                    grad_i = (Ai * distance / k - (k / distance) ** 2) @ delta
                    cache[xi.tobytes()] = (cost, grad_i)
                    return cost

                def grad_fun(x):
                    cost_fun(x)
                    return cache[x.tobytes()][1]

                res = line_search(cost_fun, grad_fun, pos[i], -grad)
                if res[0] is None:
                    continue
                new_pos[i] += res[0] * -grad
            pos = new_pos

            yield pos, grads

    elif method == "proposed":
        from scipy.optimize import line_search

        # make sure we have a coo_matrix representation
        try:
            A = A.tolil()
        except AttributeError:
            A = (sp.sparse.coo_array(A)).tolil()

        desc_dirs = np.zeros((nnodes, dim))
        conj_dirs = np.zeros((nnodes, dim))
        last_desc_dirs = np.zeros((nnodes, dim))
        last_conj_dirs = np.zeros((nnodes, dim))
        alphas = np.zeros(nnodes)

        np.random.seed(0)

        # todo fixed nodes
        vertices = np.arange(nnodes)
        for iteration in range(iterations):
            print(f"{iteration=}")
            desc_dirs.fill(0.0)
            conj_dirs.fill(0.0)
            for i in range(nnodes):
                Ai = A.getrow(i).toarray().flatten()
                delta = pos[i] - pos
                assert np.all(delta[i] == 0.0), pos
                distance = np.linalg.norm(delta, axis=1)
                distance = np.where(distance < 1e-5, 1e-5, distance)
                desc_dirs[i] = -1 * ((Ai * distance / k - (k / distance) ** 2) @ delta)
                beta = (
                    np.dot(desc_dirs[i], desc_dirs[i] - last_desc_dirs[i])
                    / np.sum(last_desc_dirs[i] ** 2)
                    if iteration > 0
                    else 0.0
                )
                beta = np.clip(beta, 0.0, 1.0)
                conj_dirs[i] = desc_dirs[i] + beta * last_conj_dirs[i]

            yield pos.copy(), conj_dirs.copy()

            alphas.fill(0.0)
            np.random.shuffle(vertices)
            alpha_mean = 0.1
            cnt = 0
            for i in vertices:
                # line search for alpha
                cache = (-1.0, 0.0, 0.0)

                Ai = A.getrow(i).toarray().flatten()
                delta_pos = pos[i] - pos
                delta_pred = conj_dirs[i] - conj_dirs
                delta_pred_2 = np.sum(delta_pred**2, axis=1)
                inner_prod = np.sum(delta_pos * delta_pred, axis=1)

                def cost_fun_i(alpha):
                    nonlocal cache
                    if alpha == cache[0]:
                        return cache[1]
                    delta = delta_pos + alpha * delta_pred
                    distance = np.linalg.norm(delta, axis=1)
                    distance = np.where(distance < 1e-5, 1e-5, distance)
                    cost = np.sum(
                        Ai * distance**3 / (3 * k) - (k**2) * np.log(distance)
                    )
                    cost_grad = Ai * distance**2 / k - k**2 / distance
                    cost_grad[i] = 0.0
                    norm_grad = (inner_prod + alpha * delta_pred_2) / distance
                    grad = np.dot(cost_grad, norm_grad)
                    cache = (alpha, cost, grad)
                    return cost

                def grad_fun_i(alpha):
                    cost_fun_i(alpha)
                    return cache[2]

                old_fval = cost_fun_i(0.0)
                gfk = grad_fun_i(0.0)
                if gfk > 0.0:
                    continue

                res, _fc, _gc, _new_fval, _old_fval, new_slope = line_search(
                    cost_fun_i,
                    grad_fun_i,
                    0.0,
                    alpha_mean,
                    gfk,
                    old_fval,
                    c1=1e-4,
                    c2=0.4,
                    maxiter=10,
                )

                if res is not None:
                    assert res >= 0 and alphas[i] >= 0.0
                    assert np.isclose(cache[0], res * alpha_mean)
                    alphas[i] = res * alpha_mean
                    if cnt == 0:
                        alpha_mean = 0
                    cnt += 1
                    alpha_mean = (alpha_mean * (cnt - 1) + alphas[i]) / cnt

            pos += alphas[:, None] * conj_dirs
            print(f"{iteration=}, {cost(pos, A, k)=}")
            last_desc_dirs = desc_dirs.copy()
            last_conj_dirs = conj_dirs.copy()

        yield pos, np.zeros((nnodes, dim))
        print(f"{cost(pos, A, k)=}")

    else:
        raise ValueError()

In [25]:
from src.python.vis.visAnimate import visAnimate


if True:
    matrixName = "jagmesh1"
    mat = getMatrixByName(matrixName)
else:
    n = 20
    matrixName = f"circle{n}"
    mat = np.zeros((n, n))
    for i in range(n):
        mat[i, (i + 1) % n] = 1
        mat[(i + 1) % n, i] = 1
    mat = scipy.sparse.csr_matrix(mat)


if scipy.sparse.issparse(mat):
    mat.setdiag(0)
    mat.eliminate_zeros()
    mat.data = np.abs(mat.data)
else:
    mat[np.diag_indices_from(mat)] = 0
    mat.data = np.abs(mat.data)

G = nx.Graph(mat)

if True:
    methodName = "proposed"
    optResults = list(spring_layout(G, seed=0, method=methodName, iterations=20))
    visAnimate(G, optResults, matrixName, methodName)
else:
    import time

    t0 = time.perf_counter()
    list(spring_layout(G, seed=0, method="L-BFGS-B", iterations=30))
    t1 = time.perf_counter()
    # list(spring_layout(G, seed=0, method="CG", iterations=10))
    t2 = time.perf_counter()
    list(spring_layout(G, seed=0, method="proposed", iterations=30))
    t3 = time.perf_counter()

    print(f"L-BFGS-B: {t1-t0}[sec]")
    # print(f"      CG: {t2-t1}[sec]")
    print(f"    myCG: {t3-t2}[sec]")

iteration=0
iteration=0, cost(pos, A, k)=np.float64(2260.3627958947745)
iteration=1
iteration=1, cost(pos, A, k)=np.float64(2101.1913022925833)
iteration=2
iteration=2, cost(pos, A, k)=np.float64(1616.612713336849)
iteration=3
iteration=3, cost(pos, A, k)=np.float64(1429.5920716291173)
iteration=4
iteration=4, cost(pos, A, k)=np.float64(1276.8240615900033)
iteration=5
iteration=5, cost(pos, A, k)=np.float64(1150.5175989438972)
iteration=6
iteration=6, cost(pos, A, k)=np.float64(1058.5541866253059)
iteration=7
iteration=7, cost(pos, A, k)=np.float64(968.2011349432498)
iteration=8
iteration=8, cost(pos, A, k)=np.float64(884.2058821181921)
iteration=9
iteration=9, cost(pos, A, k)=np.float64(825.4610083330291)
iteration=10
iteration=10, cost(pos, A, k)=np.float64(765.7433816341189)
iteration=11
iteration=11, cost(pos, A, k)=np.float64(719.4738016909755)
iteration=12
iteration=12, cost(pos, A, k)=np.float64(672.8465426313362)
iteration=13
iteration=13, cost(pos, A, k)=np.float64(630.0569718

  0%|          | 0/21 [00:00<?, ?it/s]

c:\Users\hirok\Documents\University\FruchtermanReingoldByRandomSubspace\movie\jagmesh1_proposed.gif
